In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collect import get_data
import numpy as np
import os
import re
import dash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import collect
from dash import dash_table as dt
import plotly.express as px
import datetime as t



# Data preprocessing

# get_data()
df_old = pd.read_csv('Data/prev_data.csv')
df_stock = pd.read_csv('Data/stock.csv')
df_price = pd.read_csv('Data/price.csv')
df_remove = pd.read_csv('Data/to_remove.csv')
df_age = pd.read_csv('Data/age.csv')

df_stock = df_stock[~df_stock['true_serial'].isnull()]
df_stock.drop_duplicates(inplace=True)

df_system = df_stock[df_stock['is_main_system']==True]
df_system.loc[:,'group'] = 'system'

df_value_age = df_system.merge(df_age,on='product_id',how = 'inner')

df_compo = df_stock[df_stock['is_main_system']==False]
df_price.columns = ['product_type','type','Cout']
df_compo1 = df_compo.merge(df_price ,on='product_type',how='left')
df_system_decom = df_compo1[df_compo1['Cout'].isnull()]
df_system_decom.loc[:,'group'] = 'principal'

df_system_decom = df_system_decom.merge(df_remove,on='product_type',how = 'left')
df_system_decom.head()
df_system_decom['cost_price'] = df_system_decom['cost_price']-df_system_decom['prix']
col = list(df_system.columns)
df_system_decom = df_system_decom[col]

df_composant= df_compo1[df_compo1['Cout'].notnull()]
df_composant.loc[:,'group'] = 'composant'
df_composant.loc[:,'cost_price'] = df_composant.loc[:,'Cout']
df_composant = df_composant[col]
data = pd.concat([df_system,df_system_decom,df_composant],axis=0)

value_condition = (data['etat'].isin(['Endommagé','waste','canceled']) | data['is_archived']==True)
loss_condition = (data['etat'].isin(['Endommagé','waste','canceled']) | ~data['stock_name'].str.lower().str.startswith('zzz'))
reimburse_condition = (data['etat'].isin(['Endommagé','waste','canceled']) | ~data['stock_name'].str.lower().str.startswith('yyy'))

value_cond = (df_value_age['etat'].isin(['Endommagé','waste','canceled']) | df_value_age['is_archived']==True)
df_value_age['valeur'] = np.where(value_cond,0,df_value_age['cost_price'])

data['valeur'] = np.where(value_condition,0,data['cost_price'])
data['loss'] = np.where(loss_condition,0,data['cost_price'])
data['reimburse'] = np.where(reimburse_condition,0,data['cost_price'])

/home/babou/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/babou/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [10]:
# df = pd.read_csv('Data/prev_data.csv',low_memory=False)
# df = pd.read_csv('Data/final_data.csv',low_memory=False)

In [11]:
df_value_age.head()

,date_at,mois,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,...,serial_number,cost_price,carton_id,group,age,recieved_at,type_movement,tenant_id,bucket,valeur
0,2022-03-31 00:00:00,2022-03,47311,2019-04-04 09:08:45.256129,10032417,WowSolar Controller,Neuf,NaN,10032417,personal,...,10032417,34089.00,CTN#A000003218,system,36.733333,2019-04-04 09:08:45.394903,receiving,5,> 12 mois,34089.00
1,2022-03-31 00:00:00,2022-03,869417,2021-04-22 15:44:12.888708,745847974005507119,Powerbank component,Neuf,745.0,847974005507119,personal,...,847974005507119,NaN,NaN,system,11.766667,2021-04-22 15:44:12.980379,receiving,3,9 a 12 mois,NaN
2,2022-03-31 00:00:00,2022-03,950560,2022-03-14 19:01:47.039286,4544521430153,Oolu TV Screen 43,Neuf,454.0,4521430153,multi_manager,...,4521430153,256311.00,NaN,system,0.900000,2022-03-14 19:01:47.755384,receiving,2,< 3 mois,256311.00
3,2022-03-31 00:00:00,2022-03,924106,2021-09-21 16:38:26.697096,4521320277,Oolu TV Screen 32,Neuf,NaN,4521320277,multi_manager,...,4521320277,208690.00,NaN,system,6.700000,2021-09-21 16:38:26.800615,receiving,1,6 a 9 mois,208690.00
4,2022-03-31 00:00:00,2022-03,56909,2020-01-22 15:17:39.543211,L1901909000789,Portable light component,Neuf,NaN,L1901909000789,multi_manager,...,L1901909000789,6844.78,CTN#OMNIVOLTAIC,system,26.966667,2020-01-22 15:17:39.68976,receiving,3,> 12 mois,6844.78


In [15]:
df_age_new = df_value_age[['date_at', 'mois','bucket', 'category', 'product_type', 'country', 'valeur']].groupby(
        ['date_at', 'mois','bucket', 'product_type', 'category', 'country']).sum().reset_index()
df_age_new

,date_at,mois,bucket,product_type,category,country,valeur
0,2022-03-31 00:00:00,2022-03,3 a 6 mois,Burn Stove Xtra,accessories,Nigeria,0.00
1,2022-03-31 00:00:00,2022-03,3 a 6 mois,D Light Radio,accessories,Cameroun,1917399.00
2,2022-03-31 00:00:00,2022-03,3 a 6 mois,Oolu SolarTV32,SHS+TV,Burkina,5236758.00
3,2022-03-31 00:00:00,2022-03,3 a 6 mois,Oolu SolarTV32,SHS+TV,Mali,3998000.00
4,2022-03-31 00:00:00,2022-03,3 a 6 mois,Oolu SolarTV32,SHS+TV,Niger,775816.00
...,...,...,...,...,...,...,...
73,2022-03-31 00:00:00,2022-03,> 12 mois,WowSolarTV32,pico,Burkina,1013145.00
74,2022-03-31 00:00:00,2022-03,> 12 mois,WowSolarTV32,pico,Mali,3981439.00
75,2022-03-31 00:00:00,2022-03,> 12 mois,WowSolarTV32,pico,Nigeria,0.00
76,2022-03-31 00:00:00,2022-03,> 12 mois,WowSolarTV32,pico,Senegal,231771.00


In [17]:
# df_age_new.to_csv('Data/age_prev.csv', index=False)

In [20]:
system_value = pd.pivot_table(df_age_new, values='valeur',columns=['mois'],index=['category'],aggfunc=np.sum, fill_value=0).reset_index()
age_value = pd.pivot_table(df_age_new, values='valeur',columns=['mois'],index=['bucket'],aggfunc=np.sum, fill_value=0).reset_index()

In [21]:
system_value

mois,category,2022-03
0,SHS+TV,67374386.00
1,TV22,155465.00
2,TV43,53056800.00
3,accessories,1998869.00
4,big SHS,4931927.00
5,pico,59515122.04


In [22]:
age_value

mois,bucket,2022-03
0,3 a 6 mois,38324264.00
1,6 a 9 mois,8607406.00
2,9 a 12 mois,7346760.00
3,< 3 mois,78458205.00
4,> 12 mois,54295934.04


In [7]:
df.head()

,date_at,mois,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,...,first_main_system_id,stock_name,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id
0,2022-03-31 00:00:00,2022-03,47311,2019-04-04 09:08:45.256129,10032417,WowSolar Controller,Neuf,NaN,10032417,personal,...,NaN,halidou abdou - halidou.abdou,WowSolar60,WowSolar60,pico,Niger,False,10032417,34089.0,CTN#A000003218
1,2022-03-31 00:00:00,2022-03,505,2018-06-19 16:02:05.378547,70004907,WowSolar Controller,Endommagé,NaN,70004907,multi_manager,...,NaN,OUAGADOUGOU,NaN,WowSolar Controller,pico,Burkina,False,70004907,26558.0,CTN#A000000861
2,2022-03-31 00:00:00,2022-03,869417,2021-04-22 15:44:12.888708,745847974005507119,Powerbank component,Neuf,745.0,847974005507119,personal,...,NaN,Juliet Gbakaan - juliet_gbakaan,Sherpa Powerbank,Sherpa Powerbank,accessories,Nigeria,False,847974005507119,NaN,NaN
3,2022-03-31 00:00:00,2022-03,284323,2021-02-14 19:19:18.298639,5328554C2,SKH Lamp,Endommagé,NaN,5328554C2,multi_manager,...,22776.0,Thies,NaN,SKH Lamp,pico,Senegal,False,NaN,NaN,NaN
4,2022-03-31 00:00:00,2022-03,1285,2018-09-08 13:52:06.976218,40004208,WowSolar Controller,Endommagé,NaN,40004208,multi_manager,...,NaN,BAMAKO,NaN,WowSolar Controller,pico,Mali,False,40004208,30684.0,CTN#A000000702


In [16]:
df_new = data[['date_at','mois','category','product_type','country','valeur','loss','reimburse']].groupby(['date_at','mois','product_type','category','country']).sum().reset_index()

In [17]:
df_new['category'].unique()

array(['accessories', 'pico', 'SHS+TV', 'TV43', 'big SHS', 'TV22'],
      dtype=object)

In [18]:
df_new.to_csv('Data/prev_data.csv')
df_new.to_csv('Data/final_data.csv')

In [19]:
df_new.head()

,date_at,mois,product_type,category,country,valeur,loss,reimburse
0,2022-03-31 00:00:00,2022-03,Adapter Fan,accessories,Niger,40800.0,0.0,0.0
1,2022-03-31 00:00:00,2022-03,Adapter Fan,accessories,Nigeria,6800.0,57800.0,3400.0
2,2022-03-31 00:00:00,2022-03,Burn Stove,accessories,Nigeria,0.0,0.0,31500.0
3,2022-03-31 00:00:00,2022-03,Burn Stove Xtra,accessories,Nigeria,0.0,0.0,0.0
4,2022-03-31 00:00:00,2022-03,D Light Radio,accessories,Cameroun,1937166.0,0.0,0.0


In [20]:
# finale = pd.concat([df_old,df_new],axis=1)

In [21]:
df_new.to_csv('Data/prev_data.csv', index=False)

In [110]:
today = t.date.today()

d1 = today.strftime("%Y%m%d")

In [111]:
yesterday = t.date.today()-t.timedelta(days=1)
print(yesterday)

jour = yesterday.strftime("%Y%m%d")
mois = yesterday.strftime("%Y%m")

print(jour)
print(mois)

# path = '/home/babou/PycharmProjects/logistique/Data/data'+jour+'.csv'
path = os.path.realpath('Data')+'/data'+jour+'.csv'
print(path)

df_new.to_csv(path,index=False)

2022-04-06
20220406
202204
/home/babou/PycharmProjects/logistique/Data/data20220406.csv


In [86]:
path = os.path.realpath('Data')+'/data'+jour+'.csv'
print(path)

/home/babou/PycharmProjects/logistique/Data/data20220406.csv
